In [7]:
import pandas as pd

train_absensi = pd.read_csv('./MineToday Dataset/train/train_absensi.csv')
train_mini_project = pd.read_csv('./MineToday Dataset/train/train_mini_project.csv')
train_pendaftaran = pd.read_csv('./MineToday Dataset/train/train_pendaftaran.csv')
train_pretest_ml = pd.read_csv('./MineToday Dataset/train/train_pretest_ml.csv')
train_pretest_py = pd.read_csv('./MineToday Dataset/train/train_pretest_py.csv')
train_pretest_st = pd.read_csv('./MineToday Dataset/train/train_pretest_st.csv')
train_weekly_quiz = pd.read_csv('./MineToday Dataset/train/train_weekly_quiz.csv')

test_absensi = pd.read_csv('./MineToday Dataset/test/test_absensi.csv')
test_mini_project = pd.read_csv('./MineToday Dataset/test/test_mini_project.csv')
test_pendaftaran = pd.read_csv('./MineToday Dataset/test/test_pendaftaran.csv')
test_pretest_ml = pd.read_csv('./MineToday Dataset/test/test_pretest_ml.csv')
test_pretest_py = pd.read_csv('./MineToday Dataset/test/test_pretest_py.csv')
test_pretest_st = pd.read_csv('./MineToday Dataset/test/test_pretest_st.csv')
test_weekly_quiz = pd.read_csv('./MineToday Dataset/test/test_weekly_quiz.csv')

prediction_id = pd.read_csv('./MineToday Dataset/prediction_id.csv')
submission_example = pd.read_csv('./MineToday Dataset/submission_example.csv')  


In [8]:
test_absensi_count = test_absensi.groupby('id').size().reset_index(name='absensi_count')
test_weekly_quiz_count = test_weekly_quiz.groupby('id').size().reset_index(name='weekly_quiz_count')

In [9]:
score_map = ['id', 'Score']
def select_and_rename(df, new_score_name):
    score_map = ['id', 'Score']
    df['Score'] = df['Score'].str.split('/').str[0].str.strip().astype(int)
    return df[score_map].rename(columns={'Score': new_score_name})

train_pretest_ml_baru = select_and_rename(train_pretest_ml, 'score_ml')
train_pretest_py_baru = select_and_rename(train_pretest_py, 'score_py')
train_pretest_st_baru = select_and_rename(train_pretest_st, 'score_st')  

merged_train = train_pendaftaran.merge(train_pretest_ml_baru, on='id', how='outer') \
    .merge(train_mini_project, on='id', how='outer') \
    .merge(train_weekly_quiz, on='id', how='outer') \
    .merge(train_pretest_py_baru, on='id', how='outer') \
    .merge(train_pretest_st_baru, on='id', how='outer') 
merged_train = merged_train[['id', 'score_ml', 'score_py', 'score_st']]

test_pretest_ml_baru = select_and_rename(test_pretest_ml, 'score_ml')
test_pretest_py_baru = select_and_rename(test_pretest_py, 'score_py')
test_pretest_st_baru = select_and_rename(test_pretest_st, 'score_st')  

merged_test = test_pendaftaran.merge(test_pretest_ml_baru, on='id', how='outer') \
    .merge(test_absensi_count, on='id', how='outer') \
    .merge(test_mini_project, on='id', how='outer') \
    .merge(test_weekly_quiz_count, on='id', how='outer') \
    .merge(test_pretest_py_baru, on='id', how='outer') \
    .merge(test_pretest_st_baru, on='id', how='outer') 
merged_test = merged_test[['id', 'score_ml', 'score_py', 'score_st']]
merged_train = merged_train.fillna(0)
merged_test = merged_test.fillna(0)

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer

X = merged_train[['score_ml', 'score_py', 'score_st']].copy()

imputer = SimpleImputer(strategy='constant', fill_value=0)
X_imputed = imputer.fit_transform(X)

kmeans = KMeans(n_clusters=2, random_state=42)
clusters = kmeans.fit_predict(X_imputed)

merged_train['cluster'] = clusters

cluster_mean_scores = merged_train.groupby('cluster')[['score_ml', 'score_py', 'score_st']].mean().mean(axis=1)
lulus_cluster = cluster_mean_scores.idxmax()

merged_train['label'] = merged_train['cluster'].apply(lambda x: 'Lulus' if x == lulus_cluster else 'Tidak Lulus')
merged_train = merged_train.drop(columns=['cluster'])
print(merged_train[['id', 'score_ml', 'score_py', 'score_st', 'label']].head())


                                     id  score_ml  score_py  score_st  \
0  063c738e-30ae-49ad-877d-d97287e94231       0.0       0.0       0.0   
1  0a986dd7-a658-40a9-9d1b-d3074e036e35     100.0      90.0     100.0   
2  0aa927af-2e14-47b5-bb75-c1c700b13262      50.0      80.0      90.0   
3  0abe9114-4d57-4368-9b74-8ff8d5ab22cf      50.0      90.0      90.0   
4  0b187bd8-2c9e-4c1a-ad95-1ac60fa3f39d      40.0      80.0      90.0   

         label  
0  Tidak Lulus  
1        Lulus  
2        Lulus  
3        Lulus  
4        Lulus  


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder

X_train = merged_train[['score_ml', 'score_py', 'score_st']]
y_train = merged_train['label'].map({'Tidak Lulus': 0, 'Lulus': 1})

# Model (Random Forest)
model = RandomForestClassifier(random_state=42)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='balanced_accuracy')

print(f"Balanced Accuracy (mean CV): {scores.mean():.4f}")
print(f"Balanced Accuracy (std CV): {scores.std():.4f}")

model.fit(X_train, y_train)

test = merged_test[merged_test['id'].isin(prediction_id['ID'])].copy()
test = test.drop_duplicates(subset='id')
test = test.set_index('id').loc[prediction_id['ID']].reset_index()

X_test = test[['score_ml', 'score_py', 'score_st']].fillna(0)
pred_class = model.predict(X_test)
pred_labels = ['Lulus' if p == 1 else 'Tidak Lulus' for p in pred_class]

submission = prediction_id.copy()
submission['label'] = pred_labels

submission.to_csv("submission.csv", index=False)

Balanced Accuracy (mean CV): 0.9847
Balanced Accuracy (std CV): 0.0260


In [12]:
import pickle

with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

print("Model saved as model.pkl")

Model saved as model.pkl
